In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import langchain 
from langchain.prompts import PromptTemplate

from langchain_openai import ChatOpenAI

import time
import json

## Данные

## SambaNova

In [3]:
api_key = "PASTE API KEY HERE"

llm = ChatOpenAI(
    base_url="https://api.sambanova.ai/v1/",  
    api_key=api_key,
    streaming=True,
    model="Meta-Llama-3.1-8B-Instruct",
)


## Промпт и пайплайн

In [5]:
# Определение шаблона запроса
template = """Тебе будут даны УСЛОВИЕ ЗАДАЧИ, ОБРАЗЕЦ правильного решения, НЕПРАВИЛЬНОЕ РЕШЕНИЕ ученика, а также вердикт, выданной тест-системой.
Ты должен выступить в роли учителя: проанализировать и прокомментировать ошибки в решении простой алгоритмической задачи.
В ответе укажи исправления, которые необходимо внести.
Будь сух, уверен и немногословен, старайся уместить ответ в одно-два предложения, чем короче, тем лучше. Не используй слово "ученик"

УСЛОВИЕ ЗАДАЧИ:
{task}

ОБРАЗЕЦ правильного решения:
{correct_example}

НЕПРАВИЛЬНОЕ РЕШЕНИЕ ученика:
{student_solution}

Вердикт тест-системы:
{verdict}

НИ ЗА ЧТО НЕ ВЫДАВАЙ ПРАВИЛЬНЫЙ ВАРИАНТ ОТВЕТА, ТОЛЬКО НАМЕКАЙ.
"""

prompt = PromptTemplate(input_variables=["task",'correct_example',"student_solution", 'verdict'],
                        template=template)

llm_chain = prompt | llm 

In [6]:
data = json.load(open('sort_me_round_solutions_new.json', 'r'))

## Запуск пайплайна

код оптимизирован для повторного запуска

In [9]:
res = {}

In [ ]:
#res = json.load(open('sortme-data-Llama-405B-answers.json', 'r', encoding='utf-8'))
#sum(len(res[task_id]) for task_id in res)

In [16]:
with open('log.txt', 'w') as f:
    f.write('')

In [17]:
last_n_processed = sum(len(res[task_id]) for task_id in res)
for task in data:
    answers = []
    try:
        for i, solution in enumerate(task['solutions']):
            task_id = str(task['task_id'])
            if task_id  in res and i < len(res[task_id]):
                continue
            student_solution = solution['code']
            verdict = solution['verdict']
            
            task_text = task['description']
            correct_example = task['right_solution']
            for _ in range(10):
                try:
                    output = llm_chain.invoke({"task": task_text,
                                            'correct_example': correct_example,
                                            'student_solution': student_solution,
                                            'verdict':verdict
                                            })
                    break
                except langchain.errors.RateLimitError: 
                    time.sleep(10)
            answers.append(
                {
                'solution_id': i,
                'author_comment': output.content
            })
            #print(f'\n{i}.{output.content}')
            #print('strike now!')
            #break
            with open('log.txt', 'a') as f:
                f.write('{} - Another solution processed!\n'.format(time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())))
            time.sleep(10)
        if task_id not in res:
            res[task_id] = answers
        else:
            for ans in answers:
                res[task_id].append(ans)
    except KeyError:
        continue
    #break

In [ ]:
#загрузка ответов на случай, если пайплайн был прерван пользователем
if task_id not in res:
    res[task_id] = answers
else:
    for ans in answers:
        res[task_id].append(ans)
#подсчёт и сохранение
current_n_processed = sum(len(res[task_id]) for task_id in res)
json.dump(res, open('sortme-data-Llama-405B-answers.json', 'w', encoding='utf8'), ensure_ascii=False)
json.dump(answers, open('answers.json', 'w', encoding='utf-8'), ensure_ascii=False)
print(current_n_processed - last_n_processed)